In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
markables = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

markables.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1,[1],0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 12, 13, 14]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]",1,0,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 12, 13, 14, 15, 16, 17]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[0.0, 1.0]"
4,5,"[12, 13]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[14, 15, 16, 17, 18, 19, 4, 5, 20, 21]","[0.0, 1.0]"


In [4]:
pairs_budi = pd.read_csv("data/training/mention_pairs_budi.csv")
pairs_soon = pd.read_csv("data/training/mention_pairs_soon.csv")
pairs_gilang = pd.read_csv("data/training/mention_pairs_gilang.csv")

pairs_soon.head()

,m1_id,m2_id,is_exact_match,is_words_match,is_substring,is_abbreviation,is_appositive,is_nearest_candidate,sentence_distance,word_distance,markable_distance,is_coreference
0,3,11,0,0,0,0,0,0,2,31,8,1
1,4,11,0,0,0,0,0,0,2,29,7,0
2,5,11,0,0,0,0,0,0,2,25,6,0
3,6,11,0,0,0,0,0,0,2,16,5,0
4,7,11,0,0,0,0,0,0,1,14,4,0


In [5]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

def get_data(markable_ids):
    indices = reduce(lambda a, b: a + [b], map(lambda a: markables.index[markables['id'] == a].tolist()[0], markable_ids), [])
    data = markables.loc[indices]
    
    data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
    data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
    data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
    data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

    data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
    is_singleton = np.vstack(data.is_singleton)
    
    return data_text, data_previous_words, data_next_words, data_syntactic, is_singleton

def get_pair_data(markable_ids_1, markable_ids_2):
    text_1, prev_1, next_1, syntactic_1, is_singleton_1 = get_data(markable_ids_1)
    text_2, prev_2, next_2, syntactic_2, is_singleton_2 = get_data(markable_ids_2)
    
    return text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, is_singleton_1, is_singleton_2

def get_relation_data(mention_pairs):
    return mention_pairs[['is_exact_match', 'is_words_match', 'is_substring', 'is_abbreviation', 'is_appositive', 'is_nearest_candidate', 'sentence_distance', 'word_distance', 'markable_distance']]

# Soon-Generated

In [6]:
text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, is_singleton_11, is_singleton_21 = get_pair_data(pairs_soon.m1_id, pairs_soon.m2_id)
relation_1 = get_relation_data(pairs_soon)
label_soon = np.vstack(to_categorical(pairs_soon.is_coreference, num_classes=2))

## Build Model

### Words + Context + Syntactic

In [7]:
words_context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [8]:
words_context_syntactic_model_1 = words_context_syntactic_model_builder.create_model()

W0508 12:37:42.085573 140479392503616 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:111: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0508 12:37:43.059978 140479392503616 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:135: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0508 12:37:43.103266 140479392503616 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1253: calling VarianceScaling.__init__ (from tensorflow.python.op

In [9]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 7s 1ms/sample - loss: 0.2786 - acc: 0.8861
Epoch 2/5
5548/5548==============================] - 4s 777us/sample - loss: 0.2016 - acc: 0.9030
Epoch 3/5
5548/5548==============================] - 4s 786us/sample - loss: 0.1768 - acc: 0.9297
Epoch 4/5
5548/5548==============================] - 4s 760us/sample - loss: 0.1598 - acc: 0.9427
Epoch 5/5
5548/5548==============================] - 4s 754us/sample - loss: 0.1405 - acc: 0.9531


In [10]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 4s 771us/sample - loss: 0.1106 - acc: 0.9596
Epoch 2/5
5548/5548==============================] - 4s 771us/sample - loss: 0.0952 - acc: 0.9695
Epoch 3/5
5548/5548==============================] - 4s 784us/sample - loss: 0.1017 - acc: 0.9726
Epoch 4/5
5548/5548==============================] - 4s 777us/sample - loss: 0.0568 - acc: 0.9820
Epoch 5/5
5548/5548==============================] - 4s 787us/sample - loss: 0.0837 - acc: 0.9813


In [11]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 4s 777us/sample - loss: 0.0795 - acc: 0.9845
Epoch 2/10
5548/5548==============================] - 4s 785us/sample - loss: 0.0387 - acc: 0.9885
Epoch 3/10
5548/5548==============================] - 4s 782us/sample - loss: 0.0354 - acc: 0.9906
Epoch 4/10
5548/5548==============================] - 4s 770us/sample - loss: 0.0648 - acc: 0.9841
Epoch 5/10
5548/5548==============================] - 4s 767us/sample - loss: 0.0426 - acc: 0.9888
Epoch 6/10
5548/5548==============================] - 4s 779us/sample - loss: 0.0362 - acc: 0.9926
Epoch 7/10
5548/5548==============================] - 4s 786us/sample - loss: 0.0332 - acc: 0.9913
Epoch 8/10
5548/5548==============================] - 4s 783us/sample - loss: 0.0440 - acc: 0.9865
Epoch 9/10
5548/5548==============================] - 4s 782us/sample - loss: 0.0351 - acc: 0.9957
Epoch 10/10
5548/5548==============================] - 4s 778us/sample - loss: 0.0454 - acc: 0.9895


# Gilang-Generated

In [12]:
text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, is_singleton_12, is_singleton_22 = get_pair_data(pairs_gilang.m1_id, pairs_gilang.m2_id)
relation_2 = get_relation_data(pairs_gilang)
label_gilang = np.vstack(to_categorical(pairs_gilang.is_coreference, num_classes=2))

## Build Model

### Words + Context + Syntactic

In [13]:
words_context_syntactic_model_2 = words_context_syntactic_model_builder.create_model()

In [14]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 34s 921us/sample - loss: 0.0779 - acc: 0.9823
Epoch 2/5
36503/36503==============================] - 31s 843us/sample - loss: 0.0665 - acc: 0.9828
Epoch 3/5
36503/36503==============================] - 30s 835us/sample - loss: 0.0649 - acc: 0.9827
Epoch 4/5
36503/36503==============================] - 31s 846us/sample - loss: 0.0502 - acc: 0.9828
Epoch 5/5
36503/36503==============================] - 31s 849us/sample - loss: 0.0686 - acc: 0.9827


In [15]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 31s 860us/sample - loss: 0.0469 - acc: 0.9829
Epoch 2/5
36503/36503==============================] - 31s 839us/sample - loss: 0.0452 - acc: 0.9829
Epoch 3/5
36503/36503==============================] - 31s 860us/sample - loss: 0.0400 - acc: 0.9829
Epoch 4/5
36503/36503==============================] - 31s 852us/sample - loss: 0.0399 - acc: 0.9831
Epoch 5/5
36503/36503==============================] - 30s 833us/sample - loss: 0.0649 - acc: 0.9824


In [16]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 31s 841us/sample - loss: 0.0666 - acc: 0.9827
Epoch 2/10
36503/36503==============================] - 31s 850us/sample - loss: 0.0605 - acc: 0.9824
Epoch 3/10
36503/36503==============================] - 31s 861us/sample - loss: 0.0387 - acc: 0.9829
Epoch 4/10
36503/36503==============================] - 30s 826us/sample - loss: 0.0460 - acc: 0.9828
Epoch 5/10
36503/36503==============================] - 32s 885us/sample - loss: 0.0477 - acc: 0.9828
Epoch 6/10
36503/36503==============================] - 28s 778us/sample - loss: 0.0540 - acc: 0.9828
Epoch 7/10
36503/36503==============================] - 28s 777us/sample - loss: 0.1700 - acc: 0.9818
Epoch 8/10
36503/36503==============================] - 28s 778us/sample - loss: 0.0816 - acc: 0.9827
Epoch 9/10
36503/36503==============================] - 28s 778us/sample - loss: 0.0599 - acc: 0.9829
Epoch 10/10
36503/36503==============================] - 28s 780us/sample - loss: 

# Budi-Generated

In [17]:
text_13, text_23, prev_13, prev_23, next_13, next_23, syntactic_13, syntactic_23, is_singleton_13, is_singleton_23 = get_pair_data(pairs_budi.m1_id, pairs_budi.m2_id)
relation_3 = get_relation_data(pairs_budi)
label_budi = np.vstack(to_categorical(pairs_budi.is_coreference, num_classes=2))

## Build Model

### Words + Context + Syntactic

In [18]:
words_context_syntactic_model_3 = words_context_syntactic_model_builder.create_model()

In [19]:
words_context_syntactic_model_3.fit([text_13, text_23, prev_13, prev_23, next_13, next_23, syntactic_13, syntactic_23, relation_3], label_budi, epochs=5)
words_context_syntactic_model_3.save('models/coreference_classifiers/words_context_syntactic_budi_5.model')

Epoch 1/5
167420/167420==============================] - 145s 864us/sample - loss: 0.0528 - acc: 0.9880
Epoch 2/5
167420/167420==============================] - 142s 847us/sample - loss: 0.0527 - acc: 0.9882
Epoch 3/5
167420/167420==============================] - 145s 866us/sample - loss: 0.1048 - acc: 0.9883
Epoch 4/5
167420/167420==============================] - 143s 852us/sample - loss: 0.0906 - acc: 0.9882
Epoch 5/5
167420/167420==============================] - 143s 853us/sample - loss: 0.1092 - acc: 0.9881


In [20]:
words_context_syntactic_model_3.fit([text_13, text_23, prev_13, prev_23, next_13, next_23, syntactic_13, syntactic_23, relation_3], label_budi, epochs=5)
words_context_syntactic_model_3.save('models/coreference_classifiers/words_context_syntactic_budi_10.model')

Epoch 1/5
167420/167420==============================] - 143s 856us/sample - loss: 0.0931 - acc: 0.9883
Epoch 2/5
167420/167420==============================] - 147s 876us/sample - loss: 0.0659 - acc: 0.9884
Epoch 3/5
167420/167420==============================] - 143s 856us/sample - loss: 0.2270 - acc: 0.9881
Epoch 4/5
167420/167420==============================] - 142s 848us/sample - loss: 0.1253 - acc: 0.9883
Epoch 5/5
167420/167420==============================] - 144s 862us/sample - loss: 0.1784 - acc: 0.9883


In [21]:
words_context_syntactic_model_3.fit([text_13, text_23, prev_13, prev_23, next_13, next_23, syntactic_13, syntactic_23, relation_3], label_budi, epochs=10)
words_context_syntactic_model_3.save('models/coreference_classifiers/words_context_syntactic_budi_20.model')

Epoch 1/10
167420/167420==============================] - 144s 859us/sample - loss: 0.2594 - acc: 0.9880
Epoch 2/10
167420/167420==============================] - 146s 870us/sample - loss: 0.1272 - acc: 0.9882
Epoch 4/10
167420/167420==============================] - 144s 862us/sample - loss: 0.2033 - acc: 0.9883
Epoch 5/10
167420/167420==============================] - 144s 860us/sample - loss: 0.0612 - acc: 0.9883
Epoch 6/10
167420/167420==============================] - 143s 853us/sample - loss: 0.2545 - acc: 0.9884
Epoch 7/10
167420/167420==============================] - 146s 871us/sample - loss: 0.1343 - acc: 0.9883
Epoch 8/10
167420/167420==============================] - 143s 856us/sample - loss: 0.1077 - acc: 0.9884
Epoch 9/10
167420/167420==============================] - 144s 860us/sample - loss: 0.0988 - acc: 0.9883
Epoch 10/10
167420/167420==============================] - 146s 869us/sample - loss: 0.1814 - acc: 0.9883
